<img src="https://raw.githubusercontent.com/rbizoi/PythonFormation/main/images/e-brasil.png" width="850">

# Les imports et initialisations des variables

In [1]:
from datetime import datetime
import pandas as pd, numpy as np, os, warnings, seaborn as sns, pickle, re, unicodedata
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font1 = fm.FontProperties(size=20)
font2 = fm.FontProperties(size=24)

%matplotlib inline

if int(str(sns.__version__).split('.')[1]) > 8 : 
    plt.style.use('seaborn-v0_8-darkgrid')
else:
    plt.style.use('seaborn-darkgrid')
    
sns.set(font_scale=2)
warnings.filterwarnings(action="ignore")

## Liste fichiers de données 

In [2]:
!dir ..\donnees\ebrasil

 Le volume dans le lecteur F s'appelle Data
 Le num‚ro de s‚rie du volume est D4F7-8E12

 R‚pertoire de F:\PythonFormation\donnees\ebrasil

14/02/2024  09:47    <DIR>          .
14/02/2024  09:47    <DIR>          ..
06/10/2019  20:27         9ÿ033ÿ957 olist_customers_dataset.csv
06/10/2019  20:27        61ÿ273ÿ883 olist_geolocation_dataset.csv
06/10/2019  20:27        17ÿ654ÿ914 olist_orders_dataset.csv
06/10/2019  20:27        15ÿ438ÿ671 olist_order_items_dataset.csv
06/10/2019  20:27         5ÿ777ÿ138 olist_order_payments_dataset.csv
06/10/2019  20:27        14ÿ409ÿ007 olist_order_reviews_dataset.csv
06/10/2019  20:27         2ÿ379ÿ446 olist_products_dataset.csv
28/09/2020  07:52           177ÿ799 olist_sellers_dataset.csv
17/01/2024  08:49             2ÿ542 product_category_name_translation.csv
               9 fichier(s)      126ÿ147ÿ357 octets
               2 R‚p(s)  86ÿ205ÿ247ÿ488 octets libres


## Changement de répertoire

In [3]:
os.chdir(r"../donnees")

# DataFrame $payments$

In [4]:
donnees = pd.read_csv(os.path.join('ebrasil', 'olist_order_payments_dataset.csv'))
donnees.rename(columns={col:col.replace('payment_','') for col in donnees.columns[1:]},inplace=True)
donnees.sort_values(['order_id','sequential']).head(10)

,order_id,sequential,type,installments,value
85283,00010242fe8c5a6d1ba2dd792cb16214,1,credit_card,2,72.19
2499,00018f77f2f0320c557190d7a144bdd3,1,credit_card,3,259.83
12393,000229ec398224ef6ca0657da4fc703e,1,credit_card,5,216.87
32971,00024acbcdf0a6daa1e931b038114c75,1,credit_card,2,25.78
98711,00042b26cf59d7ce69dfabb4e55b4fd9,1,credit_card,3,218.04
50570,00048cc3ae777c65dbb7d2a0634bc1ea,1,boleto,1,34.59
35027,00054e8431b9d7675808bcb819fb4a32,1,credit_card,1,31.75
22082,000576fe39319847cbb9d288c5617fa6,1,credit_card,10,880.75
100792,0005a1a1728c9d785b8e2b08b904576c,1,credit_card,3,157.60
63353,0005f50442cb953dcd1d21e1fb923495,1,credit_card,1,65.39


In [5]:
donnees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      103886 non-null  object 
 1   sequential    103886 non-null  int64  
 2   type          103886 non-null  object 
 3   installments  103886 non-null  int64  
 4   value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [6]:
donnees.columns

Index(['order_id', 'sequential', 'type', 'installments', 'value'], dtype='object')

In [7]:
payments = donnees.pivot_table( index='order_id',columns='type',
                     values=['installments','value'],
                     fill_value=0, dropna=False)
payments.columns = [col[0].replace('installments','int')+'_'+col[1] for col in payments.columns]
payments[['int_boleto',
          'int_credit_card',
          'int_debit_card',
          'int_not_defined',
          'int_voucher']]  = payments[['int_boleto',
                                       'int_credit_card',
                                       'int_debit_card',
                                       'int_not_defined',
                                       'int_voucher']].astype('int8')

In [8]:
payments.head()

,int_boleto,int_credit_card,int_debit_card,int_not_defined,int_voucher,value_boleto,value_credit_card,value_debit_card,value_not_defined,value_voucher
order_id,,,,,,,,,,
00010242fe8c5a6d1ba2dd792cb16214,0,2,0,0,0,0.0,72.19,0.0,0.0,0.0
00018f77f2f0320c557190d7a144bdd3,0,3,0,0,0,0.0,259.83,0.0,0.0,0.0
000229ec398224ef6ca0657da4fc703e,0,5,0,0,0,0.0,216.87,0.0,0.0,0.0
00024acbcdf0a6daa1e931b038114c75,0,2,0,0,0,0.0,25.78,0.0,0.0,0.0
00042b26cf59d7ce69dfabb4e55b4fd9,0,3,0,0,0,0.0,218.04,0.0,0.0,0.0


In [9]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99440 entries, 00010242fe8c5a6d1ba2dd792cb16214 to fffe41c64501cc87c801fd61db3f6244
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   int_boleto         99440 non-null  int8   
 1   int_credit_card    99440 non-null  int8   
 2   int_debit_card     99440 non-null  int8   
 3   int_not_defined    99440 non-null  int8   
 4   int_voucher        99440 non-null  int8   
 5   value_boleto       99440 non-null  float64
 6   value_credit_card  99440 non-null  float64
 7   value_debit_card   99440 non-null  float64
 8   value_not_defined  99440 non-null  float64
 9   value_voucher      99440 non-null  float64
dtypes: float64(5), int8(5)
memory usage: 5.0+ MB


## Sauvegarde en parquet

In [10]:
payments.to_parquet('./ecommerce/payments.parquet',compression='gzip', engine='pyarrow')

In [11]:
pd.read_parquet('./ecommerce/payments.parquet').info()

<class 'pandas.core.frame.DataFrame'>
Index: 99440 entries, 00010242fe8c5a6d1ba2dd792cb16214 to fffe41c64501cc87c801fd61db3f6244
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   int_boleto         99440 non-null  int8   
 1   int_credit_card    99440 non-null  int8   
 2   int_debit_card     99440 non-null  int8   
 3   int_not_defined    99440 non-null  int8   
 4   int_voucher        99440 non-null  int8   
 5   value_boleto       99440 non-null  float64
 6   value_credit_card  99440 non-null  float64
 7   value_debit_card   99440 non-null  float64
 8   value_not_defined  99440 non-null  float64
 9   value_voucher      99440 non-null  float64
dtypes: float64(5), int8(5)
memory usage: 5.0+ MB


In [12]:
!dir ecommerce

 Le volume dans le lecteur F s'appelle Data
 Le num‚ro de s‚rie du volume est D4F7-8E12

 R‚pertoire de F:\PythonFormation\donnees\ecommerce

14/02/2024  15:37    <DIR>          .
14/02/2024  15:37    <DIR>          ..
14/02/2024  15:45         4ÿ525ÿ883 customers.parquet
14/02/2024  15:37         1ÿ122ÿ602 geolocation.parquet
14/02/2024  15:46         4ÿ425ÿ246 items.parquet
14/02/2024  15:46         4ÿ527ÿ569 items_products.parquet
14/02/2024  15:45        27ÿ443ÿ040 orders.csv
14/02/2024  15:45         9ÿ269ÿ678 orders.parquet
14/02/2024  14:59         9ÿ725ÿ278 orders_payments.parquet
14/02/2024  15:00        11ÿ010ÿ093 orders_payments_reviews_p.parquet
14/02/2024  15:46         2ÿ375ÿ117 payments.parquet
14/02/2024  15:46           963ÿ000 products.parquet
14/02/2024  14:59         7ÿ858ÿ269 reviews.parquet
14/02/2024  14:59         3ÿ201ÿ554 reviews_p.parquet
14/02/2024  15:06            91ÿ808 sellers.parquet
              13 fichier(s)       86ÿ539ÿ137 octets
               2 R

In [13]:
orders = pd.read_parquet('./ecommerce/orders.parquet')

In [14]:
donnees = orders.merge(payments,how='left',on='order_id')

In [15]:
donnees.head()

,order_id,customer_id,purchase_timestamp,approved_at,delivered_carrier,delivered_customer,estimated_delivery,status,annee,mois,...,int_boleto,int_credit_card,int_debit_card,int_not_defined,int_voucher,value_boleto,value_credit_card,value_debit_card,value_not_defined,value_voucher
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,delivered,2017,10,...,0.0,1.0,0.0,0.0,1.0,0.00,18.12,0.0,0.0,10.295
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,delivered,2018,7,...,1.0,0.0,0.0,0.0,0.0,141.46,0.00,0.0,0.0,0.000
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,delivered,2018,8,...,0.0,3.0,0.0,0.0,0.0,0.00,179.12,0.0,0.0,0.000
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,delivered,2017,11,...,0.0,1.0,0.0,0.0,0.0,0.00,72.20,0.0,0.0,0.000
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,delivered,2018,2,...,0.0,1.0,0.0,0.0,0.0,0.00,28.62,0.0,0.0,0.000


In [16]:
donnees.to_parquet('./ecommerce/orders_payments.parquet',compression='gzip', engine='pyarrow')

In [17]:
donnees.shape

(99441, 33)